In [15]:
import os
import requests
import json
import pandas as pd
import numpy as np
import alpaca_trade_api as tradeapi
import streamlit as st
import copy 
from api_caller import api_call
from dotenv import load_dotenv
from make_close_price import make_daily_close
from find_portfolio_weights import portfolio_weights
from MCForecastTools import MCSimulation

In [2]:
# Read in the csv file
testing_csv = pd.read_csv('project_1_mockdata.csv')

# Create a list of the tickers
tickers = testing_csv['Ticker'].to_list()

alpaca = api_call()

timeframe = "1Day"

# Create a start and end variables to pull data during this period
end = pd.Timestamp("2020-07-13", tz="America/New_York").isoformat()
start = pd.Timestamp("2020-07-05", tz="America/New_York").isoformat()

# Pull the stock data for our tickers and selected times
df_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = start,
    end = end
).df

def make_daily_close(pulled_df, tickers):
    
    # Create an empty list to hold the dictionaries for each day
    closing_data = []
    
    # Iterate through each unique timestamp to grab each stocks data
    for date in pulled_df.index.unique():
        
        # Create a dictionary to hold the timestamp, and each stocks closing price
        day_data = {}
        
        # Store the current timestamp that we are pulling data for
        day_data['Timestamp'] = date
        
        # Iterate through the tickers for that day, and grab each ones stock price
        for stock in tickers:
            day_data[stock] = pulled_df.loc[(pulled_df.index == date) & (pulled_df['symbol'] == stock), 'close'].item()
        
        # Append the current timestamps data to the list
        closing_data.append(day_data)
        
    # Create a dataframe from the list of closing price dictionaries
    cleaned_data = pd.DataFrame(closing_data)
    
    # Return the dataframe
    return cleaned_data

# Call the function to return the cleaned daily data for our portfolio
cleaned_df = make_daily_close(df_portfolio, tickers)
cleaned_df.tail()



,Timestamp,AMZN,GOOG,WMT,JPM
1,2020-07-07 04:00:00+00:00,3000.12,1485.18,126.95,92.32
2,2020-07-08 04:00:00+00:00,3081.11,1496.00,124.44,93.30
3,2020-07-09 04:00:00+00:00,3182.63,1510.99,127.75,91.28
4,2020-07-10 04:00:00+00:00,3200.00,1541.74,130.68,96.27
5,2020-07-13 04:00:00+00:00,3104.00,1511.34,129.52,97.65


In [3]:
def portfolio_weights(clean_data, portfolio_positions, tickers):
    x = clean_data.iloc[-1,1:].to_list()
    y = portfolio_positions["Quantity"].to_list()
    total = np.dot(x, y)
    ticker_weights = {}
    for ticker in tickers:
        stock_price = clean_data[ticker].to_list()[-1]
        stock_quantity = portfolio_positions.loc[portfolio_positions['Ticker'] == ticker, 'Quantity'].item()
        stock_position = stock_price * stock_quantity
        ticker_weights[ticker] = round(stock_position / total, 2)
    return ticker_weights

    

In [4]:
weight_of_portfolio = portfolio_weights(cleaned_df, testing_csv, tickers)

weight_of_portfolio

{'AMZN': 0.5, 'GOOG': 0.34, 'WMT': 0.08, 'JPM': 0.08}

In [5]:
weight = []
for value in weight_of_portfolio.values():
    weight.append(value)
weight

[0.5, 0.34, 0.08, 0.08]

In [22]:
def create_simulation_df(df_portfolio):
    test = {}
    for ticker in df_portfolio['symbol'].unique():
        test[ticker] = copy.deepcopy(df_portfolio.loc[df_portfolio['symbol'] == ticker])
        test[ticker].drop(columns = ['symbol'], inplace = True)
    testing_df = pd.concat([test[key] for key in test.keys()], axis=1, keys=test.keys())
    return testing_df

In [24]:
x = create_simulation_df(df_portfolio)
x

AMZN                                          \
                              open       high        low    close   volume   
timestamp                                                                    
2020-07-06 04:00:00+00:00  2934.97  3059.8800  2930.0000  3057.04  6880652   
2020-07-07 04:00:00+00:00  3058.01  3069.5500  2990.0000  3000.12  5257687   
2020-07-08 04:00:00+00:00  3021.55  3083.9700  3012.4301  3081.11  5037784   
2020-07-09 04:00:00+00:00  3116.00  3193.8799  3074.0016  3182.63  6388661   
2020-07-10 04:00:00+00:00  3191.76  3215.0000  3135.7000  3200.00  5485988   
2020-07-13 04:00:00+00:00  3251.06  3344.2900  3068.3901  3104.00  7720550   

                                                       GOOG             \
                          trade_count         vwap     open       high   
timestamp                                                                
2020-07-06 04:00:00+00:00      253596  3008.206281  1480.06  1506.5900   
2020-07-07 04:00:00+00:00      224410  3034.469771  1490.00  1516.8000   
2020-07-08 04:00:00+00:00      188712  3053.037063  1494.32  1505.8799   
2020-07-09 04:00:00+00:00      242435  3134.994624  1506.45  1522.7200   
2020-07-10 04:00:00+00:00      212670  3185.414623  1506.15  1543.8300   
2020-07-13 04:00:00+00:00      352980  3235.104744  1551.12  1577.1316   

                                      ...       JPM                         \
                                 low  ...    volume trade_count       vwap   
timestamp                             ...                                    
2020-07-06 04:00:00+00:00  1472.8600  ...  18625431      127956  94.428021   
2020-07-07 04:00:00+00:00  1483.5500  ...  25757803      147216  92.623059   
2020-07-08 04:00:00+00:00  1485.6300  ...  18762297      126424  92.635983   
2020-07-09 04:00:00+00:00  1488.0850  ...  25600919      187622  91.663501   
2020-07-10 04:00:00+00:00  1496.5400  ...  28954234      213530  94.924184   
2020-07-13 04:00:00+00:00  1505.2427  ...  32079211      242070  97.764589   

                              WMT                                    \
                             open    high     low   close    volume   
timestamp                                                             
2020-07-06 04:00:00+00:00  119.80  119.87  118.22  118.89   7231664   
2020-07-07 04:00:00+00:00  118.45  127.55  118.22  126.95  31152676   
2020-07-08 04:00:00+00:00  128.00  128.13  124.40  124.44  17405804   
2020-07-09 04:00:00+00:00  125.61  128.18  124.95  127.75  14257238   
2020-07-10 04:00:00+00:00  128.71  131.37  127.26  130.68  14752333   
2020-07-13 04:00:00+00:00  131.33  133.63  128.89  129.52  14112829   

                                                   
                          trade_count        vwap  
timestamp                                          
2020-07-06 04:00:00+00:00       73823  118.862227  
2020-07-07 04:00:00+00:00      240017  124.617704  
2020-07-08 04:00:00+00:00      168093  125.860774  
2020-07-09 04:00:00+00:00      126303  126.923580  
2020-07-10 04:00:00+00:00      125936  129.564404  
2020-07-13 04:00:00+00:00      144274  131.254985  

[6 rows x 28 columns]

In [25]:
MC_five = MCSimulation(
    portfolio_data = x,
    weights = weight,
    num_simulation = 500,
    num_trading_days = 252*5
)

MC_five.portfolio_data.head()
    

AMZN                                          \
                              open       high        low    close   volume   
timestamp                                                                    
2020-07-06 04:00:00+00:00  2934.97  3059.8800  2930.0000  3057.04  6880652   
2020-07-07 04:00:00+00:00  3058.01  3069.5500  2990.0000  3000.12  5257687   
2020-07-08 04:00:00+00:00  3021.55  3083.9700  3012.4301  3081.11  5037784   
2020-07-09 04:00:00+00:00  3116.00  3193.8799  3074.0016  3182.63  6388661   
2020-07-10 04:00:00+00:00  3191.76  3215.0000  3135.7000  3200.00  5485988   

                                                                    GOOG  \
                          trade_count         vwap daily_return     open   
timestamp                                                                  
2020-07-06 04:00:00+00:00      253596  3008.206281          NaN  1480.06   
2020-07-07 04:00:00+00:00      224410  3034.469771    -0.018619  1490.00   
2020-07-08 04:00:00+00:00      188712  3053.037063     0.026996  1494.32   
2020-07-09 04:00:00+00:00      242435  3134.994624     0.032949  1506.45   
2020-07-10 04:00:00+00:00      212670  3185.414623     0.005458  1506.15   

                                      ...        JPM                  WMT  \
                                high  ...       vwap daily_return    open   
timestamp                             ...                                   
2020-07-06 04:00:00+00:00  1506.5900  ...  94.428021          NaN  119.80   
2020-07-07 04:00:00+00:00  1516.8000  ...  92.623059    -0.028211  118.45   
2020-07-08 04:00:00+00:00  1505.8799  ...  92.635983     0.010615  128.00   
2020-07-09 04:00:00+00:00  1522.7200  ...  91.663501    -0.021651  125.61   
2020-07-10 04:00:00+00:00  1543.8300  ...  94.924184     0.054667  128.71   

                                                                         \
                             high     low   close    volume trade_count   
timestamp                                                                 
2020-07-06 04:00:00+00:00  119.87  118.22  118.89   7231664       73823   
2020-07-07 04:00:00+00:00  127.55  118.22  126.95  31152676      240017   
2020-07-08 04:00:00+00:00  128.13  124.40  124.44  17405804      168093   
2020-07-09 04:00:00+00:00  128.18  124.95  127.75  14257238      126303   
2020-07-10 04:00:00+00:00  131.37  127.26  130.68  14752333      125936   

                                                    
                                 vwap daily_return  
timestamp                                           
2020-07-06 04:00:00+00:00  118.862227          NaN  
2020-07-07 04:00:00+00:00  124.617704     0.067794  
2020-07-08 04:00:00+00:00  125.860774    -0.019772  
2020-07-09 04:00:00+00:00  126.923580     0.026599  
2020-07-10 04:00:00+00:00  129.564404     0.022935  

[5 rows x 32 columns]

In [26]:
MC_five.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.


C:\Users\meber\onedrive\uw\Project-1-\MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.983637,1.012013,0.995889,0.999710,0.975987,0.998948,1.014434,0.993402,1.011973,0.990431,...,1.052373,1.004515,1.013986,1.024391,0.976423,0.999592,1.000028,1.003708,0.980050,1.033289
2,0.971450,1.006107,1.011801,1.013822,0.975648,1.003142,1.026563,0.982003,0.995823,0.974642,...,1.043758,1.004622,1.016585,1.010254,0.955759,1.009195,1.016456,0.980477,0.982365,1.050463
3,0.976075,0.991393,1.017107,1.038799,0.970089,1.000256,1.009701,0.971795,1.004906,0.960595,...,1.057158,1.044907,1.027599,1.023809,0.969761,1.005819,1.025997,0.959556,0.985047,1.064495
4,0.973135,0.988946,1.020496,1.032835,0.989049,0.983857,0.999948,0.959220,1.011552,0.933977,...,1.053405,1.054697,1.030238,1.040667,0.977107,1.008008,1.061240,0.949339,0.989446,1.087483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256,158.742219,220.710100,112.080100,435.448750,298.236683,254.311134,517.407559,234.742294,214.162788,506.512408,...,222.400673,212.309832,196.924208,85.708002,210.182015,299.025858,40.896920,113.012348,341.483459,206.235595
1257,156.984176,217.684338,117.500309,436.970510,295.762175,260.128054,503.647410,233.179806,214.367744,506.714050,...,220.255233,210.669734,197.275873,85.616032,214.065895,297.550803,40.600927,115.629246,340.806617,210.037695
1258,160.371524,219.299610,117.987461,433.470770,292.774902,267.794370,510.233088,231.927684,218.085256,514.008720,...,220.098965,214.577500,200.157049,86.592966,217.963354,294.664125,41.444491,118.817578,346.394874,213.218007
1259,162.180923,215.949222,119.605730,432.098767,295.005065,265.739369,505.397045,230.265252,216.715948,514.784929,...,225.596679,216.509034,197.024569,88.979954,216.165808,294.951515,41.926048,119.858383,353.043585,219.052433
